In [1]:
# reference:
# https://machinelearningmastery.com/time-series-forecasting-long-short-term-memory-network-python/

from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
import numpy

# date-time parsing function for loading the dataset
def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')

# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
	df = DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = concat(columns, axis=1)
	df.fillna(0, inplace=True)
	return df

# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

# scale train and test data to [-1, 1]
def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled

# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
	new_row = [x for x in X] + [value]
	array = numpy.array(new_row)
	array = array.reshape(1, len(array))
	inverted = scaler.inverse_transform(array)
	return inverted[0, -1]

# fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, units):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(units, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
		model.reset_states()
	return model

# make a one-step forecast
def forecast_lstm(model, batch_size, X):
	X = X.reshape(1, 1, len(X))
	yhat = model.predict(X, batch_size=batch_size)
	return yhat[0,0]

'''
# load dataset
series = read_csv('shampoo-sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

# transform data to be stationary
raw_values = series.values
diff_values = difference(raw_values, 1)

# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, 1)
supervised_values = supervised.values

# split data into train and test-sets
train, test = supervised_values[0:-12], supervised_values[-12:]

# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)

# fit the model
lstm_model = fit_lstm(train_scaled, 1, 3000, 4)
# forecast the entire training dataset to build up state for forecasting
train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
lstm_model.predict(train_reshaped, batch_size=1)

# walk-forward validation on the test data
predictions = list()
for i in range(len(test_scaled)):
	# make one-step forecast
	X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
	yhat = forecast_lstm(lstm_model, 1, X)
	# invert scaling
	yhat = invert_scale(scaler, X, yhat)
	# invert differencing
	yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
	# store forecast
	predictions.append(yhat)
	expected = raw_values[len(train) + i + 1]
	print('Month=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

# report performance
rmse = sqrt(mean_squared_error(raw_values[-12:], predictions))
print('Test RMSE: %.3f' % rmse)
# line plot of observed vs predicted
pyplot.plot(raw_values[-12:])
pyplot.plot(predictions)
pyplot.show()
'''


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


"\n# load dataset\nseries = read_csv('shampoo-sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)\n\n# transform data to be stationary\nraw_values = series.values\ndiff_values = difference(raw_values, 1)\n\n# transform data to be supervised learning\nsupervised = timeseries_to_supervised(diff_values, 1)\nsupervised_values = supervised.values\n\n# split data into train and test-sets\ntrain, test = supervised_values[0:-12], supervised_values[-12:]\n\n# transform the scale of the data\nscaler, train_scaled, test_scaled = scale(train, test)\n\n# fit the model\nlstm_model = fit_lstm(train_scaled, 1, 3000, 4)\n# forecast the entire training dataset to build up state for forecasting\ntrain_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)\nlstm_model.predict(train_reshaped, batch_size=1)\n\n# walk-forward validation on the test data\npredictions = list()\nfor i in range(len(test_scaled)):\n\t# make one-step forecast\n\tX, y = test_scaled[i, 0:-1]

In [3]:
# enable access to my google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


'/content'

In [5]:
# load dataset
series = read_csv('/content/gdrive/My Drive/Colab/shampoo.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

In [16]:
# transform data to be stationary
raw_values = series.values
diff_values = difference(raw_values, interval=1)
print(raw_values)
print(diff_values)

[266.  145.9 183.1 119.3 180.3 168.5 231.8 224.5 192.8 122.9 336.5 185.9
 194.3 149.5 210.1 273.3 191.4 287.  226.  303.6 289.9 421.6 264.5 342.3
 339.7 440.4 315.9 439.3 401.3 437.4 575.5 407.6 682.  475.3 581.3 646.9]
0    -120.1
1      37.2
2     -63.8
3      61.0
4     -11.8
5      63.3
6      -7.3
7     -31.7
8     -69.9
9     213.6
10   -150.6
11      8.4
12    -44.8
13     60.6
14     63.2
15    -81.9
16     95.6
17    -61.0
18     77.6
19    -13.7
20    131.7
21   -157.1
22     77.8
23     -2.6
24    100.7
25   -124.5
26    123.4
27    -38.0
28     36.1
29    138.1
30   -167.9
31    274.4
32   -206.7
33    106.0
34     65.6
dtype: float64


In [22]:
diff_values.shift(1)
# there is a NaN, so in timeseries_to_supervised, fillna(0) is done to replace NaN with 0

0       NaN
1    -120.1
2      37.2
3     -63.8
4      61.0
5     -11.8
6      63.3
7      -7.3
8     -31.7
9     -69.9
10    213.6
11   -150.6
12      8.4
13    -44.8
14     60.6
15     63.2
16    -81.9
17     95.6
18    -61.0
19     77.6
20    -13.7
21    131.7
22   -157.1
23     77.8
24     -2.6
25    100.7
26   -124.5
27    123.4
28    -38.0
29     36.1
30    138.1
31   -167.9
32    274.4
33   -206.7
34    106.0
dtype: float64

In [20]:
# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, lag=1)
supervised_values = supervised.values

# split data into train and test-sets
train, test = supervised_values[0:-12], supervised_values[-12:]

# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)

In [21]:
train_scaled

array([[-0.15241435, -0.80037766],
       [-0.80037766,  0.04828702],
       [ 0.04828702, -0.496628  ],
       [-0.496628  ,  0.17669274],
       [ 0.17669274, -0.21607769],
       [-0.21607769,  0.1891017 ],
       [ 0.1891017 , -0.1917993 ],
       [-0.1917993 , -0.32344214],
       [-0.32344214, -0.52953871],
       [-0.52953871,  1.        ],
       [ 1.        , -0.96493121],
       [-0.96493121, -0.10709469],
       [-0.10709469, -0.39411923],
       [-0.39411923,  0.17453466],
       [ 0.17453466,  0.18856218],
       [ 0.18856218, -0.59428109],
       [-0.59428109,  0.3633666 ],
       [ 0.3633666 , -0.48152145],
       [-0.48152145,  0.26625303],
       [ 0.26625303, -0.22632857],
       [-0.22632857,  0.55813326],
       [ 0.55813326, -1.        ],
       [-1.        ,  0.26733207]])

In [ ]:
'''
# code for fit_lstm:
def fit_lstm(train, batch_size, nb_epoch, units):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(units, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
		model.reset_states()
	return model
'''

# fit the model
lstm_model = fit_lstm(train_scaled, batch_size=1, nb_epoch=3000, units=4)
# forecast the entire training dataset to build up state for forecasting
train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
lstm_model.predict(train_reshaped, batch_size=1)

In [39]:
X, y = train_scaled[:, 0:-1], train_scaled[:, -1]

In [43]:
X = X.reshape(X.shape[0], 1, X.shape[1])
X.shape

(23, 1, 1)

I don't like how machinelearningmastery did this ...